In [267]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder

##load file names
csv = './Anonymized_644.Updated_cleaned_v1.3.2.tsv'
adjacency_file = './connectivity_646.csv'
adjacency = pd.read_csv(adjacency_file, index_col = 0)

In [284]:
#extract all the node names and such
left_nodes = ['L'+n for n in adjacency.columns]
right_nodes = ['R'+n for n in adjacency.columns]
nodes = left_nodes + right_nodes
all_nodes = set(nodes)
node_to_index = {word: position for position, word in enumerate(nodes)}
ambiguous_nodes = set(['2/3','3/4','2/3/4'])

In [285]:
#helper functions
def parse_lymph_nodes(node_string):
    node_string = re.sub('L2,*','L2A, L2B,', node_string)
    node_string = re.sub('R2,*','R2A, R2B,', node_string)
    nodes = [n.strip() for n in node_string.split(',')]
    for n in nodes:
        if n in ambiguous_nodes:
            return np.NaN
    nodes = [n for n in nodes if n in all_nodes]
    return nodes if len(nodes) > 0 else np.NaN

data = pd.read_csv(csv, sep='\t' , index_col=0, 
                   usecols=['Dummy ID', 'Affected Lymph node UPPER','Feeding tube 6m', 'Aspiration rate(Y/N)'],
                   dtype = {'Affected Lymph node UPPER': str}).dropna()
data['Affected Lymph node UPPER'] = data['Affected Lymph node UPPER'].apply(parse_lymph_nodes)
data = data.dropna()
data.head(5)
data.shape

(616, 3)

In [286]:
monograms = pd.DataFrame(index = data.index, columns = nodes, dtype = np.int32).fillna(0)
for p in data.itertuples():
    for lymph_node in p._1:
        monograms.loc[p.Index, lymph_node] = 1
monograms.head(5)

,L1A,L1B,L2A,L2B,L3,L4,L5A,L5B,L6,L RPLN,R1A,R1B,R2A,R2B,R3,R4,R5A,R5B,R6,R RPLN
Dummy ID,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [287]:
bigram_set = set([])

for i, name in enumerate(adjacency.columns):
    for i2 in range(i+1, adjacency.shape[1]):
        if adjacency.iloc[i,i2] > 0:
            bigram_set.add(name + adjacency.columns[i2])
' '.join(bigram_set)

'1B3 5A5B 2B5A 34 1B2A 1A6 45B 1A1B 2A2B 36 46 35A 2A3'

In [288]:
def bigramize(v):
    #shoudl take a unilateral (left or right) matrix of affected lypmh nnodes
    assert(v.shape[1] == adjacency.shape[1])
    col_names = list(v.columns)
    clean = lambda x:  re.sub('^[LR]\s*','', x)
    bigrams = []
    for i, colname in enumerate(col_names):
        nodename = clean(colname)
        for i2 in range(i+1, v.shape[1]):
            colname2 = col_names[i2]
            if nodename + clean(colname2) in bigram_set:
                bigram_vector = v[colname].values * v[colname2].values
                bigrams.append(bigram_vector.reshape(-1,1))
    return np.hstack(bigrams)
                
            
l_bigrams = bigramize(monograms.loc[:, left_nodes])
r_bigrams = bigramize(monograms.loc[:, right_nodes])
v_nonspatial = monograms.loc[:, left_nodes].values + monograms.loc[:, right_nodes].values
v_spatial = np.hstack([v_nonspatial, l_bigrams + r_bigrams])

In [289]:
# check to see values are in [0,2]
for v in [v_nonspatial, v_spatial]:
    assert(v.max() <= 2)
    assert(v.min() >= 0)

In [290]:
#similarity functions:
def tanimoto(x,y):
    numerator = x.dot(y)
    denominator = (x.dot(x) + y.dot(y) - x.dot(y))
    if denominator > 0:
        return numerator/denominator
    return 0

def cosine(x,y):
    numerator = x.dot(y)
    denominator = np.linalg.norm(x)*np.linalg.norm(y)
    if denominator > 0:
        return numerator/denominator
    return 0

def jaccard(x,y):
    x = np.nan_to_num(x/x)
    y = np.nan_to_num(y/y)
    return tanimoto(x,y)

#creates the similarity matrix
def similarity(matrix, sim_func):
    n_patients = matrix.shape[0]
    similarities = np.zeros((n_patients, n_patients))
    for p in range(n_patients):
        for p2 in range(p+1, n_patients):
            similarities[p,p2] = sim_func(matrix[p], matrix[p2])
    similarities += similarities.transpose()
    np.fill_diagonal(similarities, 1)
    return similarities

def distance(matrix, sim_func):
    sim = similarity(matrix, sim_func)
    return 1 - ((sim - sim.max())/(sim.max() - sim.min()))

In [291]:
from sklearn.cluster import AgglomerativeClustering
from scipy.stats import chi2_contingency
    
ft = data['Feeding tube 6m'] == 'Y'
aspiration = data['Aspiration rate(Y/N)'] == 'Y'
toxicity = ft + aspiration > 0
toxicities = [ft, aspiration, toxicity]

def get_contingency_table(x, y):
        #assumes x and y are two equal length vectors, creates a mxn contigency table from them
        cols = sorted(list(np.unique(y)))
        rows = sorted(list(np.unique(x)))
        tabel = np.zeros((len(rows), len(cols)))
        for row_index in range(len(rows)):
            row_var = rows[row_index]
            for col_index in range(len(cols)):
                rowset = set(np.argwhere(x == row_var).ravel())
                colset = set(np.argwhere(y == cols[col_index]).ravel())
                tabel[row_index, col_index] = len(rowset & colset)
        return tabel
    
def get_correlation(encoding, sim_func,  n_clusters = 6, linkage = 'complete'):
    tox_names = ['feeding_tube', 'aspiration', 'either']
    ac = AgglomerativeClustering(n_clusters = n_clusters, 
                                 affinity = 'precomputed', 
                                 linkage=linkage)
    cluster_labels = ac.fit_predict(distance(encoding, sim_func))
    for tox in toxicities:
        print(tox_names.pop(0))
        contingency = get_contingency_table(cluster_labels, tox)
        print(chi2_contingency(contingency)[1])
        print(contingency)
        print()

F:\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [292]:
print('nonspatial - tanimto')
get_correlation(v_nonspatial, tanimoto)
print('nonspatial - cosine')
get_correlation(v_nonspatial, cosine)
print('nonspatial - jaccard')
get_correlation(v_nonspatial, jaccard)

print('spatial - tanimto')
get_correlation(v_spatial, tanimoto)
print('spatial - cosine')
get_correlation(v_spatial, cosine)
print('spatial - jaccard')
get_correlation(v_spatial, jaccard)

nonspatial - tanimto
feeding_tube
0.03731921915041625
[[  6.   2.]
 [208.  57.]
 [281.  44.]
 [  5.   0.]
 [ 11.   0.]
 [  1.   1.]]

aspiration
0.37185372256046123
[[  7.   1.]
 [212.  53.]
 [276.  49.]
 [  4.   1.]
 [ 11.   0.]
 [  2.   0.]]

either
0.06807330999678589
[[  5.   3.]
 [182.  83.]
 [250.  75.]
 [  4.   1.]
 [ 11.   0.]
 [  1.   1.]]

nonspatial - cosine


F:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


feeding_tube
0.09395317226023524
[[311.  54.]
 [  5.   0.]
 [178.  47.]
 [  5.   2.]
 [ 12.   0.]
 [  1.   1.]]

aspiration
0.6923789640134987
[[305.  60.]
 [  4.   1.]
 [183.  42.]
 [  7.   0.]
 [ 11.   1.]
 [  2.   0.]]

either
0.6110882393583064
[[272.  93.]
 [  4.   1.]
 [160.  65.]
 [  5.   2.]
 [ 11.   1.]
 [  1.   1.]]

nonspatial - jaccard


F:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)
F:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide
F:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


feeding_tube
0.0033098470574771683
[[449.  81.]
 [  5.   0.]
 [ 40.  20.]
 [  6.   2.]
 [ 11.   0.]
 [  1.   1.]]

aspiration
0.1233973011664917
[[445.  85.]
 [  4.   1.]
 [ 43.  17.]
 [  7.   1.]
 [ 11.   0.]
 [  2.   0.]]

either
0.005410829818324839
[[399. 131.]
 [  4.   1.]
 [ 33.  27.]
 [  5.   3.]
 [ 11.   0.]
 [  1.   1.]]

spatial - tanimto


...

F:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


feeding_tube
0.02766708800433946
[[294.  46.]
 [  6.   2.]
 [195.  55.]
 [  5.   0.]
 [ 11.   0.]
 [  1.   1.]]

aspiration
0.1806651932963005
[[291.  49.]
 [  7.   1.]
 [197.  53.]
 [  4.   1.]
 [ 11.   0.]
 [  2.   0.]]

either
0.1185982278767666
[[259.  81.]
 [  5.   3.]
 [173.  77.]
 [  4.   1.]
 [ 11.   0.]
 [  1.   1.]]

spatial - jaccard


F:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)
F:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide
F:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


feeding_tube
0.02075038716470803
[[201.  57.]
 [  5.   0.]
 [288.  44.]
 [  6.   2.]
 [ 11.   0.]
 [  1.   1.]]

aspiration
0.10026840859200924
[[202.  56.]
 [  4.   1.]
 [286.  46.]
 [  7.   1.]
 [ 11.   0.]
 [  2.   0.]]

either
0.07012495620991607
[[177.  81.]
 [  4.   1.]
 [255.  77.]
 [  5.   3.]
 [ 11.   0.]
 [  1.   1.]]



F:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


In [266]:
data.shape

(618, 3)